In [143]:
import pandas as pd
import eurostat

In [144]:
df = eurostat.get_data_df(code="HBS_STR_T223")
df = df.rename(columns={'geo\TIME_PERIOD': 'geo', '2015':'OBS_VALUE'})


In [145]:
# Keep only the rows where TIME_PERIOD is equal to 2015
df = df.loc[:, ['quantile','coicop','geo','OBS_VALUE']]


In [146]:
df_inflation = eurostat.get_data_df(code="PRC_HICP_MANR",filter_pars={'startPeriod':"2021-03"})
df_inflation = df_inflation.rename(columns={'geo\TIME_PERIOD': 'geo'})


In [147]:
merged_df = df.merge(df_inflation, on=["coicop", "geo"])


In [148]:
cols_to_multiply = [col for col in merged_df.columns if col.startswith('202')]
merged_df[['w' + col for col in cols_to_multiply]] = merged_df[cols_to_multiply].apply(lambda x: x * merged_df['OBS_VALUE']/1000)


In [149]:
merged_df_5_chars = merged_df[merged_df['coicop'].str.len() == 5]


In [150]:
columns_to_sum = [col for col in merged_df_5_chars.columns if col.startswith('w')]
                  
summed_columns = []
for col in columns_to_sum:
    new_col_name = col[1:]
    summed_columns.append(new_col_name)
    merged_df_5_chars[new_col_name] = merged_df_5_chars.groupby(['geo', 'quantile'])[col].transform('sum')

df_inflation_by_quintile = merged_df_5_chars[['geo', 'quantile'] + summed_columns].drop_duplicates()

C:\Users\maxen\AppData\Local\Temp\ipykernel_988\899747291.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_5_chars[new_col_name] = merged_df_5_chars.groupby(['geo', 'quantile'])[col].transform('sum')
C:\Users\maxen\AppData\Local\Temp\ipykernel_988\899747291.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_5_chars[new_col_name] = merged_df_5_chars.groupby(['geo', 'quantile'])[col].transform('sum')
C:\Users\maxen\AppData\Local\Temp\ipykernel_988\899747291.py:7: SettingWithCopyWarning

In [151]:
# For EU27_2020, only the groups of coicop with 4 characters have value

merged_df_4_chars = merged_df[merged_df['coicop'].str.len() == 4]
merged_df_4_chars = merged_df_4_chars.loc[(merged_df_4_chars['geo'] == 'EU27_2020')]
summed_columns = []
for col in columns_to_sum:
    new_col_name = col[1:]
    summed_columns.append(new_col_name)
    merged_df_4_chars[new_col_name] = merged_df_4_chars.groupby(['geo', 'quantile'])[col].transform('sum')

df_inflation_by_quintile_eu27 = merged_df_4_chars[['geo', 'quantile'] + summed_columns].drop_duplicates()


In [152]:
df_inflation_by_quintile = pd.concat([df_inflation_by_quintile, df_inflation_by_quintile_eu27])

In [153]:
# we drop Italy and EA because they don't have the values, and TR because they have a crazy high inflation
df_inflation_by_quintile = df_inflation_by_quintile.loc[~df_inflation_by_quintile['geo'].isin(['IT', 'EA', 'TR'])]

In [154]:
value_vars = df_inflation_by_quintile.filter(regex='^202').columns


In [155]:
df_inflation_by_quintile_melt = df_inflation_by_quintile.melt(id_vars=["geo", "quantile"], value_vars=value_vars, var_name="month", value_name="inflation")

In [156]:
df_pivot = df_inflation_by_quintile_melt.pivot_table(index=['geo', 'month'], columns='quantile', values='inflation').reset_index()

df_pivot['Q1/Q5'] = df_pivot['QUINTILE1'] / df_pivot['QUINTILE5']

quintile_columns = ['QUINTILE1', 'QUINTILE2', 'QUINTILE3', 'QUINTILE4', 'QUINTILE5']
quintiles_df = df_pivot[quintile_columns]

df_pivot['normalized_std'] = quintiles_df.std(axis=1) / quintiles_df.mean(axis=1)

df_pivot['Q1-Q5'] = df_pivot['QUINTILE1'] - df_pivot['QUINTILE5']
df_pivot['(Q1-Q5)/Q1'] = (df_pivot['QUINTILE1'] - df_pivot['QUINTILE5'])/df_pivot['QUINTILE1']

df_inflation_by_quintile = df_pivot.reset_index(drop=True)
df_inflation_by_quintile = df_inflation_by_quintile.drop(["UNK"], axis=1)

# Energy inflation

In [158]:
df_inflation_NRG = df_inflation[df_inflation["coicop"] == "NRG"]

In [159]:
df_inflation_NRG = df_inflation_NRG.melt(id_vars=["geo"], value_vars=value_vars, var_name="month", value_name="inflationNRG")

In [160]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(df_inflation_NRG, on=["geo", "month"])

# Food inflation

In [196]:
df_inflation_FOOD = df_inflation[df_inflation["coicop"] == "FOOD"]

In [197]:
df_inflation_FOOD = df_inflation_FOOD.melt(id_vars=["geo"], value_vars=value_vars, var_name="month", value_name="inflationFOOD")

In [198]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(df_inflation_FOOD, on=["geo", "month"])

# Gini

In [161]:
gini = eurostat.get_data_df(code="TESSI190")

In [162]:
import numpy as np

def get_most_recent(row):
    for year in range(2022, 2010, -1):
        if not pd.isnull(row[str(year)]):
            return row[str(year)]
    return np.nan

gini["most_recent"] = gini.apply(get_most_recent, axis=1)
gini = gini.rename(columns={"geo\TIME_PERIOD": "geo"})
gini = gini.loc[:, ["geo", "most_recent"]]
gini = gini.rename(columns={"most_recent": "gini"})


In [163]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(gini, on="geo")

# Price levels

In [164]:
price_levels_energy = eurostat.get_data_df(code="prc_ppp_ind", filter_pars={'startPeriod':"2021", "endPeriod":"2021", "ppp_cat":["A010405"], "na_item":"PLI_EU27_2020"})
price_levels_energy = price_levels_energy.rename(columns={"geo\TIME_PERIOD":"geo", "2021": "energy_price_level"})
price_levels_energy = price_levels_energy.loc[:, ["geo", "energy_price_level"]]



In [165]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(right=price_levels_energy, on="geo")

In [166]:
price_levels_transport = eurostat.get_data_df(code="prc_ppp_ind", filter_pars={'startPeriod':"2021", "endPeriod":"2021", "ppp_cat":["A0107"], "na_item":"PLI_EU27_2020"})
price_levels_transport = price_levels_transport.rename(columns={"geo\TIME_PERIOD":"geo", "2021": "transport_price_level"})
price_levels_transport = price_levels_transport.loc[:, ["geo", "transport_price_level"]]

In [167]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(right=price_levels_transport, on="geo")

# GDP per capita

In [168]:
code = "NAMA_10_PC"
gdp = eurostat.get_data_df(code=code, filter_pars={'startPeriod':"2021", "endPeriod":"2021", "unit": "CP_EUR_HAB", "na_item":"B1GQ"})
gdp = gdp.rename(columns={"geo\TIME_PERIOD":"geo", "2021": "gdp"})
gdp = gdp.loc[:, ["geo", "gdp"]]

In [169]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(right=gdp, on="geo")

# Government expenditure

In [170]:
code = "GOV_10A_EXP"
gov_exp = eurostat.get_data_df(code=code, filter_pars={'startPeriod':"2021", "endPeriod":"2021", "na_item":"TE", "sector":"S13", "unit":"PC_GDP", "cofog99": "TOTAL"})
gov_exp = gov_exp.rename(columns={"geo\TIME_PERIOD":"geo", "2021": "gov_exp"})
gov_exp = gov_exp.loc[:, ["geo", "gov_exp"]]

In [171]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(right=gov_exp, on="geo")

In [172]:
code = "GOV_10A_EXP"
gov_exp_social = eurostat.get_data_df(code=code, filter_pars={'startPeriod':"2021", "endPeriod":"2021", "na_item":"TE", "sector":"S13", "unit":"PC_GDP", "cofog99": "GF10"})
gov_exp_social = gov_exp_social.rename(columns={"geo\TIME_PERIOD":"geo", "2021": "gov_exp_social"})
gov_exp_social = gov_exp_social.loc[:, ["geo", "gov_exp_social"]]

In [173]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(right=gov_exp_social, on="geo")

# Urbanisation

In [174]:
code = "ILC_LVHO01"
urbanisation = eurostat.get_data_df(code=code, filter_pars={"endPeriod":"2022", "incgrp":"TOTAL", "building": "TOTAL", "deg_urb": "DEG1"})

urbanisation["most_recent"] = urbanisation.apply(get_most_recent, axis=1)


urbanisation = urbanisation.rename(columns={"geo\TIME_PERIOD":"geo", "most_recent": "urbanisation"})
urbanisation = urbanisation.loc[:, ["geo", "urbanisation"]]

In [175]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(urbanisation, on="geo")

# Share of electric consumption in final energy consumption

In [190]:
code = "NRG_IND_FECF"
electric_share = eurostat.get_data_df(code=code, filter_pars={"startPeriod":"2021", "endPeriod":"2021", "nrg_bal":"FC_E","siec":"E7000"})



electric_share = electric_share.rename(columns={"geo\TIME_PERIOD":"geo", "2021": "electric_share"})
electric_share = electric_share.loc[:, ["geo", "electric_share"]]

In [191]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(electric_share, on="geo")

# At-risk-of-poverty rate before social transfers

In [213]:
code = "ILC_LI10"
poverty = eurostat.get_data_df(code=code, filter_pars={"startPeriod":"2011", "endPeriod":"2022", "age": "total", "indic_il":"LI_R_MD60BT", "sex":"T"})

poverty["most_recent"] = poverty.apply(get_most_recent, axis=1)


poverty = poverty.rename(columns={"geo\TIME_PERIOD":"geo", "most_recent": "poverty"})
poverty = poverty.loc[:, ["geo", "poverty"]]

In [214]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(poverty, on="geo")

# Import

In [218]:
code = "TET00004"
importation = eurostat.get_data_df(code=code, filter_pars={"startPeriod":"2011", "endPeriod":"2021"})


importation = importation.rename(columns={"geo\TIME_PERIOD":"geo", "2021": "importation"})
importation = importation.loc[:, ["geo", "importation"]]

In [219]:
df_inflation_by_quintile = df_inflation_by_quintile.merge(importation, on="geo")

# Analysis

In [192]:
df_inflation_by_quintile = df_inflation_by_quintile.dropna()

In [177]:
df_inflation_by_quintile.describe()

,QUINTILE1,QUINTILE2,QUINTILE3,QUINTILE4,QUINTILE5,Q1/Q5,normalized_std,Q1-Q5,(Q1-Q5)/Q1,inflationNRG,gini,energy_price_level,transport_price_level,gdp,gov_exp,gov_exp_social,urbanisation
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,7.271781,7.112183,7.043678,6.962606,6.775261,1.006081,0.071568,0.496520,-0.021545,25.014429,29.189286,89.735714,95.275000,35475.714286,47.167857,17.114286,37.653571
std,5.741714,5.399576,5.135835,4.924312,4.636832,1.088487,0.101829,1.518120,0.795889,18.960473,3.836458,20.901571,18.372198,23768.248231,6.894051,4.032479,9.980004
min,-1.012000,-0.981100,-1.033400,-1.056200,-1.206500,-21.647059,-1.293301,-2.862700,-5.299442,-28.200000,21.800000,47.600000,69.100000,10330.000000,24.800000,8.700000,18.900000
25%,2.846775,2.936775,3.144425,3.237025,3.288650,0.911086,0.030149,-0.400250,-0.094847,12.100000,26.750000,74.750000,82.050000,18292.500000,43.475000,13.575000,31.025000
50%,6.115050,5.995350,5.995900,6.018200,5.922450,1.043885,0.053085,0.246450,0.043403,22.050000,29.250000,92.000000,91.050000,26090.000000,48.050000,17.600000,37.600000
75%,9.875950,9.391025,9.201350,9.155200,8.880950,1.155246,0.092687,0.987125,0.135949,35.075000,31.925000,103.650000,107.225000,45257.500000,50.925000,20.600000,43.325000
max,34.990300,31.546600,27.866700,25.908300,23.498800,16.727099,0.780795,11.491500,17.398876,113.800000,38.400000,130.400000,138.500000,112780.000000,59.000000,24.800000,59.600000


In [178]:
df_inflation_by_quintile.dtypes

geo                       object
month                     object
QUINTILE1                float64
QUINTILE2                float64
QUINTILE3                float64
QUINTILE4                float64
QUINTILE5                float64
Q1/Q5                    float64
normalized_std           float64
Q1-Q5                    float64
(Q1-Q5)/Q1               float64
inflationNRG             float64
gini                     float64
energy_price_level       float64
transport_price_level    float64
gdp                      float64
gov_exp                  float64
gov_exp_social           float64
urbanisation             float64
dtype: object

In [179]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


Y = df_inflation_by_quintile['Q1/Q5']*100
X = df_inflation_by_quintile.drop(['Q1/Q5', 'normalized_std', 'geo','month', 'QUINTILE1'], axis=1)
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Q1/Q5   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     2.692
Date:                Mon, 08 May 2023   Prob (F-statistic):           0.000735
Time:                        01:32:14   Log-Likelihood:                -4257.0
No. Observations:                 700   AIC:                             8544.
Df Residuals:                     685   BIC:                             8612.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   127.04

In [180]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


Y = df_inflation_by_quintile['normalized_std']
X = df_inflation_by_quintile.drop(['Q1/Q5', 'normalized_std', 'geo','month', 'QUINTILE1'], axis=1)
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         normalized_std   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     6.957
Date:                Mon, 08 May 2023   Prob (F-statistic):           1.70e-13
Time:                        01:32:14   Log-Likelihood:                 652.89
No. Observations:                 700   AIC:                            -1276.
Df Residuals:                     685   BIC:                            -1208.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.39

In [181]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


Y = df_inflation_by_quintile['inflationNRG']
X = df_inflation_by_quintile['transport_price_level']
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           inflationNRG   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     15.24
Date:                Mon, 08 May 2023   Prob (F-statistic):           0.000104
Time:                        01:32:14   Log-Likelihood:                -3044.8
No. Observations:                 700   AIC:                             6094.
Df Residuals:                     698   BIC:                             6103.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    10.64

In [182]:
Y = df_inflation_by_quintile['QUINTILE1']
X = df_inflation_by_quintile[['inflationNRG', 'gini', 'gov_exp_social', 'urbanisation']]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              QUINTILE1   R-squared:                       0.455
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     144.9
Date:                Mon, 08 May 2023   Prob (F-statistic):           4.80e-90
Time:                        01:32:14   Log-Likelihood:                -2003.9
No. Observations:                 700   AIC:                             4018.
Df Residuals:                     695   BIC:                             4041.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              5.3305      1.571      3.

In [183]:
Y = df_inflation_by_quintile['QUINTILE5']
X = df_inflation_by_quintile[['inflationNRG', 'gini', 'gov_exp_social', 'urbanisation']]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              QUINTILE5   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.401
Method:                 Least Squares   F-statistic:                     118.0
Date:                Mon, 08 May 2023   Prob (F-statistic):           8.13e-77
Time:                        01:32:14   Log-Likelihood:                -1885.1
No. Observations:                 700   AIC:                             3780.
Df Residuals:                     695   BIC:                             3803.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              6.6410      1.326      5.

In [223]:
Y = df_inflation_by_quintile['Q1-Q5']
X = df_inflation_by_quintile[['inflationNRG', 'inflationFOOD', 'gini', 'gov_exp_social', 'urbanisation', 'electric_share', 'poverty']]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Q1-Q5   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     116.0
Date:                Mon, 08 May 2023   Prob (F-statistic):          3.33e-112
Time:                        02:15:02   Log-Likelihood:                -1013.3
No. Observations:                 700   AIC:                             2043.
Df Residuals:                     692   BIC:                             2079.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.8288      0.437     -6.

In [207]:
Y = df_inflation_by_quintile['Q1-Q5']
X = df_inflation_by_quintile[['gdp']]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Q1-Q5   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.6344
Date:                Mon, 08 May 2023   Prob (F-statistic):              0.426
Time:                        02:02:10   Log-Likelihood:                -1284.7
No. Observations:                 700   AIC:                             2573.
Df Residuals:                     698   BIC:                             2582.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4282      0.103      4.151      0.0

In [186]:
Y = df_inflation_by_quintile['(Q1-Q5)/Q1']
X = df_inflation_by_quintile[['inflationNRG', 'gini', 'gov_exp_social', 'urbanisation']]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             (Q1-Q5)/Q1   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     5.698
Date:                Mon, 08 May 2023   Prob (F-statistic):           0.000163
Time:                        01:33:04   Log-Likelihood:                -821.66
No. Observations:                 700   AIC:                             1653.
Df Residuals:                     695   BIC:                             1676.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.0398      0.290     -3.

In [225]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'sklearn'